In [10]:

import pandas as pd
import numpy as np
from datetime import datetime
#画图
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['Arial Unicode MS']
mpl.rcParams['axes.unicode_minus']=False
#获取数据
import yfinance as yf 


In [11]:
# Mock get_indicators function for testing (Replace this with your actual implementation)
def get_indicators(code):
    # Mock data for testing
    data = pd.DataFrame({
        'tr_yoy': np.random.uniform(5, 15, 10),  # 营业收入增长率
        'op_yoy': np.random.uniform(10, 30, 10),  # 营业利润增长率
        'grossprofit_margin': np.random.uniform(20, 40, 10),  # 毛利率
        'expense_of_sales': np.random.uniform(5, 15, 10),  # 期间费用率
        'inv_turn': np.random.uniform(1, 3, 10),  # 存货周转率
        'ocfps': np.random.uniform(0, 5, 10),  # 每股经营性现金流
        'eps': np.random.uniform(0, 5, 10),  # 每股收益
        'roe_yearly': np.random.uniform(10, 20, 10),  # 年化净资产收益率
        'roa2_yearly': np.random.uniform(5, 10, 10),  # 总资产收益率
        'pb': np.random.uniform(1, 5, 10),  # 市净率
        'pe_ttm': np.random.uniform(10, 30, 10),  # 动态市盈率
        'netprofit_yoy': np.random.uniform(10, 20, 10)  # 盈利增长率
    })
    return data


In [12]:
#存在缺失值或者负值（市盈率）情况下评分直接为0
#营业收入增长率打分（0-10）
def cal_tryoy(y):
    '''y是营业收入增长率'''
    try:
        return 5+ min(round(y-10),5) if y>=10 else 5+ max(round(y-10),-5)
    except:
        return 0
#营业利润增长率打分(0-10)
def cal_opyoy(y):
    '''y是营业利润增长率'''
    try:
        return 5+ min(round((y-20)/2),5) if y>=20 else 5+ max(round((y-20)/2),-5)
    except:
        return 0
#毛利率打分
def cal_gpm(y):
    '''y是最近季度毛利率-前三季度平均毛利率'''
    try:
        return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
    except:
        return 0
#期间费用率打分
def cal_exp(y):
    '''y是最近季度期间费用率-前三季度平均期间费用率'''
    try:
        return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
    except:
        return 0
#存货周转率打分
def cal_inv(y):
    '''y是（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100'''
    try:
        return 5+min(round(y/2),5) if y>0 else max(round(y/2),-5)+5
    except:
        return 0
#每股经营性现金流打分
def cal_ocfp(y):
    '''y是（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100'''
    try:
        return 5+min(round(y/4),5) if y>0 else max(round(y/4),-5)+5
    except:
        return 0
#净资产收益率打分
def cal_roe(y):
    '''y是年化净资产收益率'''
    try:
        return 5+ min(round(y-15),5) if y>=15 else 5+ max(round(y-15),-5)
    except:
        return 0
#总资产报酬率打分
def cal_roa(y):
    '''y是最近季度年化总资产报酬率'''
    try:
        return min(round((y-5)/0.5),10) if y>=5 else max(round(y-5),0)
    except:
        return 0
#市净率打分
def cal_pb(y):
    '''y是市净率'''
    try:
        return 5-max(round((y-3)/0.4),-5) if y<=3 else 5-min(round((y-3)/0.4),5)
    except:
        return 0
#动态市盈率相对盈利增长率（PEG）打分
def cal_pe(y):
    '''y是动态市盈率相对盈利增长率'''
    try:
        return 5-max(round((y-1)/0.1),-5) if y<=1 else 5-min(round((y-1)/0.1),5)
    except:
        return 0

In [13]:
#存在缺失值或者负值（市盈率）情况下评分直接为0
#营业收入增长率打分（0-10）
def cal_tryoy(y):
    '''y是营业收入增长率'''
    try:
        return 5+ min(round(y-10),5) if y>=10 else 5+ max(round(y-10),-5)
    except:
        return 0
#营业利润增长率打分(0-10)
def cal_opyoy(y):
    '''y是营业利润增长率'''
    try:
        return 5+ min(round((y-20)/2),5) if y>=20 else 5+ max(round((y-20)/2),-5)
    except:
        return 0
#毛利率打分
def cal_gpm(y):
    '''y是最近季度毛利率-前三季度平均毛利率'''
    try:
        return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
    except:
        return 0
#期间费用率打分
def cal_exp(y):
    '''y是最近季度期间费用率-前三季度平均期间费用率'''
    try:
        return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
    except:
        return 0
#存货周转率打分
def cal_inv(y):
    '''y是（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100'''
    try:
        return 5+min(round(y/2),5) if y>0 else max(round(y/2),-5)+5
    except:
        return 0
#每股经营性现金流打分
def cal_ocfp(y):
    '''y是（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100'''
    try:
        return 5+min(round(y/4),5) if y>0 else max(round(y/4),-5)+5
    except:
        return 0
#净资产收益率打分
def cal_roe(y):
    '''y是年化净资产收益率'''
    try:
        return 5+ min(round(y-15),5) if y>=15 else 5+ max(round(y-15),-5)
    except:
        return 0
#总资产报酬率打分
def cal_roa(y):
    '''y是最近季度年化总资产报酬率'''
    try:
        return min(round((y-5)/0.5),10) if y>=5 else max(round(y-5),0)
    except:
        return 0
#市净率打分
def cal_pb(y):
    '''y是市净率'''
    try:
        return 5-max(round((y-3)/0.4),-5) if y<=3 else 5-min(round((y-3)/0.4),5)
    except:
        return 0
#动态市盈率相对盈利增长率（PEG）打分
def cal_pe(y):
    '''y是动态市盈率相对盈利增长率'''
    try:
        return 5-max(round((y-1)/0.1),-5) if y<=1 else 5-min(round((y-1)/0.1),5)
    except:
        return 0

In [14]:
# 指标得分计算
def indicator_score(code):
    data = get_indicators(code)

    # 如果 data 没有 date 列，使用 data.index 或者生成日期
    if 'date' not in data.columns:
        data['date'] = data.index  # 如果 data.index 是日期类型
        # 或者生成日期列，例如从 2020-01-01 开始，每个数据点增加一天
        # data['date'] = pd.date_range(start='2020-01-01', periods=len(data), freq='D')

    # 计算各项指标得分
    data['营收得分'] = data['tr_yoy'].apply(cal_tryoy)
    data['利润得分'] = data['op_yoy'].apply(cal_opyoy)
    data['gpm'] = data['grossprofit_margin'] - data['grossprofit_margin'].rolling(3).mean()
    data['毛利得分'] = data['gpm'].apply(cal_gpm)
    data['exp'] = data['expense_of_sales'] - data['expense_of_sales'].rolling(3).mean()
    data['费用得分'] = data['exp'].apply(cal_exp)
    data['inv'] = (data['inv_turn'] - data['inv_turn'].rolling(3).mean()) * 100 / data['inv_turn'].rolling(3).mean()
    data['周转得分'] = data['inv'].apply(cal_inv)
    data['ocf'] = (data['ocfps'].rolling(3).sum() - data['eps'].rolling(3).sum()) * 100 / data['eps'].rolling(3).sum()
    data['现金得分'] = data['ocf'].apply(cal_ocfp)
    data['净资产得分'] = data['roe_yearly'].apply(cal_roe)
    data['总资产得分'] = data['roa2_yearly'].apply(cal_roa)
    data['市净率得分'] = data['pb'].apply(cal_pb)
    data['peg'] = data['pe_ttm'] / data['netprofit_yoy'].rolling(3).mean()
    data['市盈率得分'] = data['peg'].apply(cal_pe)

    # 计算总分
    data['总分'] = data[['营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                    '总资产得分', '市净率得分', '市盈率得分']].sum(axis=1)

    # 返回数据框，并确保包含日期列
    return data[['date', '营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                 '总资产得分', '市净率得分', '市盈率得分', '总分']]

In [15]:
def indicator_score(code):
    data = get_indicators(code)

    # 如果 data 没有 date 列，使用 data.index 或者生成日期
    if 'date' not in data.columns:
        data['date'] = data.index  # 如果 data.index 是日期类型
        # 或者生成日期列，例如从 2020-01-01 开始，每个数据点增加一天
        # data['date'] = pd.date_range(start='2020-01-01', periods=len(data), freq='D')

    # 计算各项指标得分
    data['营收得分'] = data['tr_yoy'].apply(cal_tryoy)
    data['利润得分'] = data['op_yoy'].apply(cal_opyoy)
    data['gpm'] = data['grossprofit_margin'] - data['grossprofit_margin'].rolling(3).mean()
    data['毛利得分'] = data['gpm'].apply(cal_gpm)
    data['exp'] = data['expense_of_sales'] - data['expense_of_sales'].rolling(3).mean()
    data['费用得分'] = data['exp'].apply(cal_exp)
    data['inv'] = (data['inv_turn'] - data['inv_turn'].rolling(3).mean()) * 100 / data['inv_turn'].rolling(3).mean()
    data['周转得分'] = data['inv'].apply(cal_inv)
    data['ocf'] = (data['ocfps'].rolling(3).sum() - data['eps'].rolling(3).sum()) * 100 / data['eps'].rolling(3).sum()
    data['现金得分'] = data['ocf'].apply(cal_ocfp)
    data['净资产得分'] = data['roe_yearly'].apply(cal_roe)
    data['总资产得分'] = data['roa2_yearly'].apply(cal_roa)
    data['市净率得分'] = data['pb'].apply(cal_pb)
    data['peg'] = data['pe_ttm'] / data['netprofit_yoy'].rolling(3).mean()
    data['市盈率得分'] = data['peg'].apply(cal_pe)

    # 计算总分
    data['总分'] = data[['营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                    '总资产得分', '市净率得分', '市盈率得分']].sum(axis=1)

    # 返回数据框，并确保包含日期列
    return data[['date', '营收得分', '利润得分', '毛利得分', '费用得分', '周转得分', '现金得分', '净资产得分', 
                 '总资产得分', '市净率得分', '市盈率得分', '总分']]

In [16]:
 # 测试 TSMC 的指标得分
code = '2330.TW'
result = indicator_score(code)
print(result)

/var/folders/_z/wf028lv51rl4_zw8pxj4k3740000gn/T/ipykernel_88972/1608334184.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pb'].fillna(method='ffill', inplace=True)
/var/folders/_z/wf028lv51rl4_zw8pxj4k3740000gn/T/ipykernel_88972/1608334184.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['pb'].fillna(method='ffill', inplace=True)


KeyError: 'tr_yoy'